In [1]:
# 权重参数调整
import torch
path_src = '/home/hjf/workspace/mmpretrain/work_dirs/pretrained_in1k/03segformer/mit_b0_mmlab.pth'
path_out = '/home/hjf/workspace/mmpretrain/work_dirs/pretrained_in1k/03segformer/mit_b0_mmlab_heads_1248.pth'
params = torch.load(path_src, map_location='cpu')
if 'state_dict' in params.keys():
    params = params['state_dict']
params_out = {}

# 获取尺寸不匹配的参数信息
path_logs = '/home/hjf/workspace/mmpretrain/work_dirs/pretrained_in1k/03segformer/debug_loadlog.txt'
logs = []
with open(path_logs, 'r', encoding='utf-8') as f:
    for line in f.readlines():
        line = line.strip()
        if line.startswith('size mismatch for '):
            logs.append(line)
# print(logs)
params_mismatch = {}
for log in logs:
    # print(log)
    index_start_of_key = len('size mismatch for ')
    index_end_of_key = log.find(':')
    key = log[index_start_of_key:index_end_of_key]

    index_start_shape1 = log.find('shape torch.Size([') + len('shape torch.Size([')
    index_end_shape1 = log.find(']) from checkpoint,')
    shape1 = log[index_start_shape1:index_end_shape1]
    shape1 = shape1.split(', ')
    shape1 = [int(x) for x in shape1]
    # print(shape1)

    index_start_shape2 = log.find('model is torch.Size([') + len('model is torch.Size([')
    index_end_shape2 = log.find(']).')
    shape2 = log[index_start_shape2:index_end_shape2]
    shape2 = shape2.split(', ')
    shape2 = [int(x) for x in shape2]
    # print(shape2)
    params_mismatch[key] = [shape1, shape2]
# print(params_mismatch)

# 开始修改
for key in params.keys():
    if key in params_mismatch.keys():
        # print(key)
        value = params[key]
        shape1, shape2 = params_mismatch[key]
        if len(shape1) == 1:
            value_new = value[:shape2[0]]
        elif len(shape1) == 2:
            value_new = value[:shape2[0], :shape2[1]]
        elif len(shape1) == 3:
            value_new = value[:shape2[0], :shape2[1], :shape2[2]]
        elif len(shape1) == 4:
            value_new = value[:shape2[0], :shape2[1], :shape2[2], :shape2[3]]
        else:
            value_new = value
            print(key)
        params_out[key] = value_new
    else:
        params_out[key] = params[key]
torch.save(params_out, path_out)
print('Done!')

Done!
